In [106]:
import numpy as np
from math import sqrt, exp

In [107]:
contract="S"
underlying=100.0
strike=100.0
life=365
vol=0.3
rf=0.03
div=0
cp=1
american=True
steps=5

In [108]:
def payoffFunc(und, strk, cp):
    return max((und - strk) * cp, 0)

In [109]:
h = life / 365 / steps
u = exp(vol * sqrt(h))   #upFactor
d = exp(-vol * sqrt(h))  #DownFactor
drift = 1 if (contract == "F") else exp(rf * h)

q = (drift - d) / (u - d)
qx = 1 - q

payoff = lambda x, y, cp: max((x/1.0 - y/1.0)*cp, 0)
payoff = np.vectorize(payoff)
undTree=[]
#optTree=[]

In [110]:
#build undTree
for i in range(1,steps+1):
    # Number of prices per period
    period_prices = []
    for j in range(i+1):
        up_factor_repeat = i-j
        down_factor_repeat = j

        values = np.append(np.repeat(u, up_factor_repeat), np.repeat(d, down_factor_repeat))
        assetAtNode = values.prod() * underlying
        period_prices.append(assetAtNode)
    undTree.append(period_prices)

In [111]:
print (undTree)
print (len(undTree))

[[114.35804413868395, 87.44465748183686], [130.77762259225187, 100.0, 76.46568122115768], [149.5547313675689, 114.35804413868395, 87.44465748183686, 66.86515303499459], [171.0278657088147, 130.77762259225187, 100.0, 76.46568122115768, 58.470004046157065], [195.58412215673542, 149.5547313675689, 114.35804413868395, 87.44465748183686, 66.86515303499459, 51.1288947677782]]
5


In [112]:
bound=undTree[steps-1]
print(bound)

[195.58412215673542, 149.5547313675689, 114.35804413868395, 87.44465748183686, 66.86515303499459, 51.1288947677782]


In [113]:
#boundary=[]
boundary=payoff((undTree[steps-1]),strike,cp)

In [114]:
#payoff No Anda
print(boundary)
print(payoff(bound,strike,1))
print(payoff(bound,strike,-1))

[95.58412216 49.55473137 14.35804414  0.          0.          0.        ]
[95.58412216 49.55473137 14.35804414  0.          0.          0.        ]
[ 0  0  0 12 33 48]


In [115]:
# l la medida del ultimo nivel es siempre steps+1
l=len(undTree[steps-1])
opt=np.zeros(l)
print(l)
print(len(undTree))

6
5


In [116]:
#Boundary Conditions
optPrices=[]
optTree=[]
for i in range(l):
    
    optPrices.append(payoffFunc(undTree[steps-1][i],strike,-1))
 
print("optPrice",optPrices)
optTree.append(optPrices)
print("optTree",optTree)

optPrice [0, 0, 0, 12.555342518163144, 33.13484696500541, 48.8711052322218]
optTree [[0, 0, 0, 12.555342518163144, 33.13484696500541, 48.8711052322218]]


In [121]:
lastLevel=optTree
print("Medida Lastlevel :",len(lastLevel[0]))
print(len(optTree))
#print(lastLevel)

Medida Lastlevel : 6
1


In [119]:
print(optTree[0])

[0, 0, 0, 12.555342518163144, 33.13484696500541, 48.8711052322218]
